In [1]:
import numpy as np
import pandas as pd

import rasterio
import geopandas as gpd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

from shapely.geometry import mapping
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from rasterio.plot import reshape_as_raster, reshape_as_image

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.model_selection import cross_val_score
from rasterio.enums import Resampling
from rasterio.warp import reproject

In [2]:
def exctractPointValueFromTiffWithShp(shp, tif, cls, out):
    shapefile  = gpd.read_file(shp)
    Test_values = []
    Test_target = []
    # 打开栅格文件
    with rasterio.open(tif) as src:
        # 获取栅格文件的地理转换信息
        transform = src.transform
        # 将 Shapefile 中的点数据转换为栅格坐标
        shapefile['raster_x'], shapefile['raster_y'] = ~transform * (shapefile['geometry'].x, shapefile['geometry'].y)
        # 读取栅格文件中点数据位置的像素值
        for idx, row in shapefile.iterrows():
            px = int(row['raster_x'])
            py = int(row['raster_y'])
            if 0 <= px < src.width and 0 <= py < src.height:
                # 读取所有波段的像素值
                pixel_values = src.read(window=((py, py+1), (px, px+1)))
                # print("Values at point {}: {}".format(idx, pixel_values))
                # print(pixel_values[6][0][0])
                
                ndvi = (pixel_values[3] - pixel_values[2]) / (pixel_values[3] + pixel_values[2])
                ndvi[ndvi == np.inf] = 0
                ndvi = np.nan_to_num(ndvi)
                
                value_a = np.append(pixel_values[0:6].squeeze(), ndvi[0][0])
                Test_values.append(value_a)
                Test_target.append(cls)
    # 创建一个示例数组，这里假设数据是随机的
    data_y = Test_target
    data = Test_values

    # 将数据转换为 Pandas DataFrame
    df = pd.DataFrame(data)
    df_y = pd.DataFrame(data_y)
    df["class"] = df_y

    # 保存为 Excel 文件
    df.to_excel(f'{out}.xlsx', index=False)
        

In [ ]:
shp = './Grass-legume_ground_points/Grass_points_paddock-9.shp'
tif = 'All_Paddock_26_JUL_2024_ortho_bgrent.tiff'
cls = 'grass'
out = 'grass_pd9'
exctractPointValueFromTiffWithShp(shp=shp, tif=tif, cls=cls, out=out)

In [15]:
shp = '../Modeling_data_09-22-2024/Grass-legume-ground_points/Paddock_9-Legume.shp'
tif = '../Modeling_data_09-22-2024/All_Paddock_22_SEP_2024_ortho_bgrent.tiff'
cls = 'legume'
out = '../Modeling_data_09-22-2024/pad9_Legume'
exctractPointValueFromTiffWithShp(shp=shp, tif=tif, cls=cls, out=out)

In [66]:
# add gps in table
def exctractPointValue_GPS_FromTiffWithShp(shp, tif, cls, out):
    shapefile  = gpd.read_file(shp)
    Test_values = []
    Test_target = []
    # 打开栅格文件
    with rasterio.open(tif) as src:
        # 获取栅格文件的地理转换信息
        transform = src.transform
        # 将 Shapefile 中的点数据转换为栅格坐标
        shapefile['raster_x'], shapefile['raster_y'] = ~transform * (shapefile['geometry'].x, shapefile['geometry'].y)
        lon_list = shapefile['geometry'].x
        lat_list = shapefile['geometry'].y
        # 读取栅格文件中点数据位置的像素值
        for idx, row in shapefile.iterrows():
            px = int(row['raster_x'])
            py = int(row['raster_y'])
            if 0 <= px < src.width and 0 <= py < src.height:
                # 读取所有波段的像素值
                pixel_values = src.read(window=((py, py+1), (px, px+1)))
                # print("Values at point {}: {}".format(idx, pixel_values))
                # print(pixel_values[6][0][0])
                
                ndvi = (pixel_values[3] - pixel_values[2]) / (pixel_values[3] + pixel_values[2])
                ndvi[ndvi == np.inf] = 0
                ndvi = np.nan_to_num(ndvi)
                
                value_a = np.append(pixel_values[0:6].squeeze(), ndvi[0][0])
                Test_values.append(value_a)
                Test_target.append(cls)
    # 创建一个示例数组，这里假设数据是随机的
    data_y = Test_target
    data = Test_values

    # 将数据转换为 Pandas DataFrame
    df = pd.DataFrame(data)
    df_y = pd.DataFrame(data_y)
    df["class"] = df_y
    df["lon"] = lon_list
    df["lat"] = lat_list

    # 保存为 Excel 文件
    df.to_csv(f'{out}_gps.csv', index=False)


In [80]:
# add gps in table
def exctractPointValue_GPS_FromTiffWithShp(shp, tif, cls, out):
    shapefile  = gpd.read_file(shp)
    print(shapefile.crs)
    Test_values = []
    Test_target = []
    # 打开栅格文件
    with rasterio.open(tif) as src:
        # 获取栅格文件的地理转换信息
        transform = src.transform
        tiff_crs = src.crs
        if shapefile.crs != src.crs:
            print("crs is not same")
            shapefile = shapefile.to_crs(tiff_crs)
        else:
            print("same crs")
        
        # 将 Shapefile 中的点数据转换为栅格坐标
        shapefile['raster_x'], shapefile['raster_y'] = ~transform * (shapefile['geometry'].x, shapefile['geometry'].y)
        lon_list = shapefile['geometry'].x
        lat_list = shapefile['geometry'].y
        print(lon_list)
        # 读取栅格文件中点数据位置的像素值
        for idx, row in shapefile.iterrows():
            px = int(row['raster_x'])
            py = int(row['raster_y'])
            if 0 <= px < src.width and 0 <= py < src.height:
                # 读取所有波段的像素值
                pixel_values = src.read(window=((py, py+1), (px, px+1)))
                # print("Values at point {}: {}".format(idx, pixel_values))
                # print(pixel_values[6][0][0])
                
                ndvi = (pixel_values[3] - pixel_values[2]) / (pixel_values[3] + pixel_values[2])
                ndvi[ndvi == np.inf] = 0
                ndvi = np.nan_to_num(ndvi)
                
                value_a = np.append(pixel_values[0:6].squeeze(), ndvi[0][0])
                Test_values.append(value_a)
                Test_target.append(cls)
    # 创建一个示例数组，这里假设数据是随机的
    data_y = Test_target
    data = Test_values

    # 将数据转换为 Pandas DataFrame
    df = pd.DataFrame(data)
    df_y = pd.DataFrame(data_y)
    df["class"] = df_y
    df["lon"] = lon_list
    df["lat"] = lat_list

    # 保存为 Excel 文件
    df.to_csv(f'{out}_gps.csv', index=False)


In [87]:
shp = './Grass-legume_ground_points/Legume_points_paddock9.shp'
tif = './All_Paddock_26_JUL_2024_ortho_bgrent.tiff'
cls = 'legume'
out = './pointTable/pad9_legume'
exctractPointValue_GPS_FromTiffWithShp(shp=shp, tif=tif, cls=cls, out=out)

epsg:3857
crs is not same
0    -85.187094
1    -85.187201
2    -85.187125
3    -85.187092
4    -85.187199
5    -85.187277
6    -85.187432
7    -85.187502
8    -85.187410
9    -85.187594
10   -85.187973
11   -85.186791
dtype: float64


In [88]:
shp = '../Modeling_data_09-22-2024/Grass-legume-ground_points/Bare_soil.shp'
tif = '../Modeling_data_09-22-2024/All_Paddock_22_SEP_2024_ortho_bgrent.tiff'
cls = 'soil'
out = '../Modeling_data_09-22-2024/soil'
exctractPointValue_GPS_FromTiffWithShp(shp=shp, tif=tif, cls=cls, out=out)

epsg:4326
same crs
0    -85.187565
1    -85.188460
2    -85.188470
3    -85.187661
4    -85.187548
5    -85.187652
6    -85.187761
7    -85.187512
8    -85.188375
9    -85.186578
10   -85.187592
11   -85.187605
12   -85.187480
13   -85.187472
14   -85.188330
15   -85.188398
16   -85.188415
17   -85.188362
18   -85.188342
19   -85.188294
20   -85.188441
21   -85.188469
22   -85.187599
23   -85.187557
dtype: float64


In [2]:
import random
# Define the range from 1 to 16
numbers = list(range(1, 17))

# Randomly select 80% of the numbers (80% of 16 is 12)
train_samples = random.sample(numbers, int(0.8 * len(numbers)))

# Label each number as 'train' if it's in train_samples, otherwise label as 'test'
labels = {num: 'train' if num in train_samples else 'test' for num in numbers}
print(train_samples)
print(labels)

[1, 10, 11, 4, 3, 5, 2, 8, 14, 9, 16, 7]
{1: 'train', 2: 'train', 3: 'train', 4: 'train', 5: 'train', 6: 'test', 7: 'train', 8: 'train', 9: 'train', 10: 'train', 11: 'train', 12: 'test', 13: 'test', 14: 'train', 15: 'test', 16: 'train'}
